In [1]:
from langchain_core.messages.tool import ToolOutputMixin

In [2]:
import pinecone
import torch
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from transformers import AutoModel, AutoTokenizer
import os

c:\Users\vishw\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import os
os.environ["MISTRAL_API_KEY"] = "4y9r3Ftj2aRO3SQN3NemBark2zBPmNaj"
os.environ["PINECONE_API_KEY"] = "pcsk_6ZnvJ4_LaaNJkoEP2q2ynCynb2eevEgncBitui8JGky9c5zr9GwcRizZyenNzj6mryY7xw"
os.environ["PINECONE_ENV"] = "us-east-1"

In [4]:
# Initialize Pinecone
pc = pinecone.Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "sknai"

In [5]:
tokenizer = AutoTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")
model = AutoModel.from_pretrained("monologg/biobert_v1.1_pubmed")

def embed_text(text):
    """Generate BioBERT embeddings for a given text."""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().tolist() 

In [6]:
vector_store = Pinecone(
    index=pc.Index(index_name),  # Use the Pinecone client to access the index
    embedding=embed_text,        # Use the embedding function
    text_key="text"              # Assuming 'text' is the key for the documents
)

C:\Users\vishw\AppData\Local\Temp\ipykernel_9836\1600786067.py:1: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vector_store = Pinecone(
c:\Users\vishw\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [7]:
query_embedding = embed_text("Symptoms of enzecma")
# Assuming your Pinecone index is accessible via pc.Index(index_name),
# you can query it directly:
pinecone_results = pc.Index(index_name).query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)

print("\nDirect Pinecone Query Results:")
for match in pinecone_results["matches"]:
    print("Score:", match["score"])
    print("Metadata:", match.get("metadata"))


Direct Pinecone Query Results:
Score: 0.797986448
Metadata: {'chunk_id': '784', 'text': 'gradually heals with routine care.One other rare complication only occurs with the largest ofcapillary angiomas.Blood platelets become sequestered in the abnor- mal vascular channels of the angioma,creating a consumption coagulopathy and uncontrolled bleeding (Katzenbach–Merritt syndrome).The bleeding can be dealt Figure 12.20 Small strawberry naevus on the with by administration ofsystemic steroids. left side of the nose. CAVERNOUS HAEMANGIOMA This is a soft, compressible, mauvish-blue'}
Score: 0.79529649
Metadata: {'chunk_id': '1063', 'text': 'dreadful facial scarring,with hirsutes.This combi- nation ofclinical features has suggested to some that these patients provoked the fable of‘werewolves’. 261 Metabolic disorders and reticulohistiocytic proliferative disorders HAEMOCHROMATOSIS (BRONZED DIABETES) There are primary and symptomatic forms ofthis disorder.In the primary form, there is excessive

In [8]:
# Setup retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 100})

import getpass
import os

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")
    
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0.4,
    max_retries=2,
   
)


In [14]:
prompt = ChatPromptTemplate.from_messages([
    ("system", " Identified diseasae is ezecma. Answer the questions user asked for the same using your own knowdegde and the information provided. Use simpler terms and act like you are explaining to a patient "),
    ("human", "{question}")
])
# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Stuffing retrieved docs into the prompt
    retriever=retriever,
    return_source_documents=True
)

In [15]:
from langchain_core.messages.tool import ToolOutputMixin


In [11]:
query = "What is bacterial infection?"
similar_docs = vector_store.similarity_search(query, k=3)  # top 3 similar documents

print("Retrieved Documents:")
for i, doc in enumerate(similar_docs, start=1):
    print(f"\nDocument {i}:")
    print("Content:", doc.page_content)  # or doc.content if that's your field name
    print("Metadata:", doc.metadata)

Retrieved Documents:

Document 1:
Content: gradually heals with routine care.One other rare complication only occurs with the largest ofcapillary angiomas.Blood platelets become sequestered in the abnor- mal vascular channels of the angioma,creating a consumption coagulopathy and uncontrolled bleeding (Katzenbach–Merritt syndrome).The bleeding can be dealt Figure 12.20 Small strawberry naevus on the with by administration ofsystemic steroids. left side of the nose. CAVERNOUS HAEMANGIOMA This is a soft, compressible, mauvish-blue
Metadata: {'chunk_id': '784'}

Document 2:
Content: skin is put together and how it functions in health.Although,at first glance,skin may appear quite complicated to the uniniti- ated, a slightly deeper look shows that there is a kind of elegant logic about its architecture,which is directed to subserving vital functions. THE SKIN SURFACE The skin surface is the delineation between living processes and the potentially injurious outside world and has not only a 

In [16]:
# Test with a user query
query = "What are treatments of Eczema?" 
result = qa_chain({"query": query})

# Print the response
print("AI Response:", result["result"])

# Print sources
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata['source']}")


AI Response: Based on the provided context, treatments for eczema include:

1. **Emollients**: These are moisturizing treatments that help to reduce water loss and restore the skin's barrier function. They are a mainstay of treatment and should be applied frequently.

2. **Topical Corticosteroids**: These are anti-inflammatory agents that can help reduce inflammation and symptoms of eczema. They come in different potencies.

3. **Topical Immunomodulators**: These are non-steroidal anti-inflammatory agents, such as tacrolimus and pimecrolimus, which can be used for short-term and intermittent long-term treatment of eczema.

4. **Systemic Therapies**: In severe cases, systemic therapies like oral corticosteroids, cyclosporine, methotrexate, azathioprine, and dupilumab (a biologic) may be used.

5. **Phototherapy**: This involves the use of ultraviolet light to treat eczema.

6. **Antihistamines**: These can help relieve itching, although they do not treat the underlying cause of eczema.


KeyError: 'source'